In [14]:
import findspark
findspark.init()

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from gatubelxs.spark_utils import SparkUtils
from gatubelxs.streaming_monitor import StreamingPerformanceMonitor, analyze_performance_data

In [16]:
import os 
base_path = "/home/jovyan/notebooks/data"

for topic in ["page_views", "click_events", "user_interactions"]:
    os.makedirs(f"{base_path}/{topic}/checkpoint", exist_ok=True)
    os.makedirs(f"{base_path}/{topic}", exist_ok=True)  # For the data files too

In [17]:
spark = SparkSession.builder \
    .appName("FinalProjectGatubelxs") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [18]:
performance_monitor = StreamingPerformanceMonitor()
spark.streams.addListener(performance_monitor)

In [20]:
TOPICS = ["page_views", "click_events", "user_interactions"]
KAFKA_BOOTSTRAP_SERVER = "kafka:9093"

In [21]:
page_views_schema = SparkUtils.generate_schema(
    [

        ("user_id", "string"),
        ("session_id", "string"),
        ("page_url", "string"),
        ("referrer_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string")
    ]
)

In [22]:
click_events_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("element_id", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string"),
        ("x_coord", "float"),
        ("y_coord", "float"),
    ]
)

In [23]:
user_interaction_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("interaction_type", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("details", "string"),
        ("timestamp", "string")
    ]
)

In [24]:
def create_stream_query(topic, schema, output_dir):
    kafka__stream = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
        .option("subscribe", topic) \
        .load()
    
    parsed_data = kafka__stream.selectExpr("CAST(value as STRING) as json") \
        .withColumn("data", from_json(col("json"), schema)) \
        .select("data.*")

    return parsed_data.writeStream \
        .format("parquet") \
        .outputMode("append") \
        .option("path", f"{base_path}/{topic}") \
        .option("checkpointLocation", f"{base_path}/{topic}/checkpoint") \
        .trigger(processingTime="5 seconds") \
        .start()


In [25]:
queries = [
    create_stream_query("page_views", page_views_schema, "./datalake/page_views"),
    create_stream_query("click_events", click_events_schema, "./datalake/click_events"),
    create_stream_query("user_interactions", user_interaction_schema, "./datalake/user_interactions"),
]

for q in queries:
    q.awaitTermination(timeout=60)

for q in queries:
    q.stop()

analyze_performance_data(performance_monitor.metrics_history)

25/05/13 03:35:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 03:35:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 03:35:12 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 03:35:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 03:35:12 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Query started: 6d1873b1-444e-4aed-9f88-f985e0238522 at 2025-05-13 03:35:12.377300
Query started: 6d1873b1-444e-4aed-9f88-f985e0238522 at 2025-05-13 03:35:12.379197
Query started: 26e943bf-4ad6-455b-8e7e-9017a2def1d9 at 2025-05-13 03:35:12.432300
Query started: 26e943bf-4ad6-455b-8e7e-9017a2def1d9 at 2025-05-13 03:35:12.432910
Query started: 181d51de-4ee4-4850-9f0b-14b43949beab at 2025-05-13 03:35:12.476702
Query started: 181d51de-4ee4-4850-9f0b-14b43949beab at 2025-05-13 03:35:12.477874


25/05/13 03:35:12 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.



Performance summary for page_views]:
Processed 1689 rows at 1457.29 rows/second
Input rate: 0.00 rows/second
Processing time: 1159ms

Performance summary for page_views]:
Processed 1689 rows at 1457.29 rows/second
Input rate: 0.00 rows/second
Processing time: 1159ms



Performance summary for click_events]:
Processed 1541 rows at 896.45 rows/second
Input rate: 0.00 rows/second
Processing time: 1719ms

Performance summary for click_events]:
Processed 1541 rows at 896.45 rows/second
Input rate: 0.00 rows/second
Processing time: 1719ms

Performance summary for user_interactions]:
Processed 1560 rows at 722.89 rows/second
Input rate: 0.00 rows/second
Processing time: 2158ms

Performance summary for user_interactions]:
Processed 1560 rows at 722.89 rows/second
Input rate: 0.00 rows/second
Processing time: 2158ms

Performance summary for click_events]:
Processed 13 rows at 17.74 rows/second
Input rate: 5.06 rows/second
Processing time: 733ms

Performance summary for click_events]:
Processed 13 rows at 17.74 rows/second
Input rate: 5.06 rows/second
Processing time: 733ms

Performance summary for user_interactions]:
Processed 5 rows at 3.87 rows/second
Input rate: 1.98 rows/second
Processing time: 1293ms

Performance summary for user_interactions]:
Processe

...


Performance summary for click_events]:
Processed 3 rows at 2.41 rows/second
Input rate: 0.60 rows/second
Processing time: 1247ms

Performance summary for click_events]:
Processed 3 rows at 2.41 rows/second
Input rate: 0.60 rows/second
Processing time: 1247ms

Performance summary for user_interactions]:
Processed 11 rows at 6.20 rows/second
Input rate: 2.20 rows/second
Processing time: 1775ms

Performance summary for user_interactions]:
Processed 11 rows at 6.20 rows/second
Input rate: 2.20 rows/second
Processing time: 1775ms



Performance summary for user_interactions]:
Processed 10 rows at 14.62 rows/second
Input rate: 2.00 rows/second
Processing time: 684ms

Performance summary for user_interactions]:
Processed 10 rows at 14.62 rows/second
Input rate: 2.00 rows/second
Processing time: 684ms

Performance summary for page_views]:
Processed 13 rows at 10.66 rows/second
Input rate: 2.60 rows/second
Processing time: 1219ms

Performance summary for page_views]:
Processed 13 rows at 10.66 rows/second
Input rate: 2.60 rows/second
Processing time: 1219ms

Performance summary for click_events]:
Processed 13 rows at 7.37 rows/second
Input rate: 2.60 rows/second
Processing time: 1765ms

Performance summary for click_events]:
Processed 13 rows at 7.37 rows/second
Input rate: 2.60 rows/second
Processing time: 1765ms

Performance summary for user_interactions]:
Processed 15 rows at 72.12 rows/second
Input rate: 3.00 rows/second
Processing time: 208ms

Performance summary for user_interactions]:
Processed 15 rows at 72.1


Performance summary for click_events]:
Processed 13 rows at 7.35 rows/second
Input rate: 2.60 rows/second
Processing time: 1768ms

Performance summary for click_events]:
Processed 13 rows at 7.35 rows/second
Input rate: 2.60 rows/second
Processing time: 1768ms

Performance summary for page_views]:
Processed 16 rows at 68.09 rows/second
Input rate: 3.20 rows/second
Processing time: 235ms

Performance summary for page_views]:
Processed 16 rows at 68.09 rows/second
Input rate: 3.20 rows/second
Processing time: 235ms

Performance summary for click_events]:
Processed 10 rows at 12.79 rows/second
Input rate: 2.00 rows/second
Processing time: 782ms

Performance summary for click_events]:
Processed 10 rows at 12.79 rows/second
Input rate: 2.00 rows/second
Processing time: 782ms

Performance summary for user_interactions]:
Processed 16 rows at 12.12 rows/second
Input rate: 3.20 rows/second
Processing time: 1320ms

Performance summary for user_interactions]:
Processed 16 rows at 12.12 rows/seco


Performance summary for click_events]:
Processed 19 rows at 27.86 rows/second
Input rate: 3.80 rows/second
Processing time: 682ms

Performance summary for click_events]:
Processed 19 rows at 27.86 rows/second
Input rate: 3.80 rows/second
Processing time: 682ms

Performance summary for page_views]:
Processed 14 rows at 11.35 rows/second
Input rate: 2.80 rows/second
Processing time: 1234ms

Performance summary for page_views]:
Processed 14 rows at 11.35 rows/second
Input rate: 2.80 rows/second
Processing time: 1234ms

Performance summary for user_interactions]:
Processed 22 rows at 17.15 rows/second
Input rate: 4.40 rows/second
Processing time: 1283ms

Performance summary for user_interactions]:
Processed 22 rows at 17.15 rows/second
Input rate: 4.40 rows/second
Processing time: 1283ms
Query terminated: 6d1873b1-444e-4aed-9f88-f985e0238522 at 2025-05-13 03:38:12.506758
Query terminated: 6d1873b1-444e-4aed-9f88-f985e0238522 at 2025-05-13 03:38:12.509261
Query terminated: 26e943bf-4ad6-45

Query terminated: 181d51de-4ee4-4850-9f0b-14b43949beab at 2025-05-13 03:38:12.527004
Query terminated: 181d51de-4ee4-4850-9f0b-14b43949beab at 2025-05-13 03:38:12.534378


In [ ]:
sc.stop()